In [ ]:
'''referring EMAprep file for cleaning up for analysis to schedule re-notification'''

In [1]:
import pandas as pd
import json
import math
import numpy as np
import datetime
import mysql
from mysql import connector
import logging

In [2]:
# Obtain all device_id from aware database
# Note: the user and password are hardcoded now

test_db = mysql.connector.connect(
      host="localhost",
      user="root",
      passwd="xuxuhai1996", 
      database="uwexp_staff_raw"
    )
sql_cmd = "SELECT * FROM aware_device"
df_aware_device = pd.read_sql(sql_cmd, test_db)
device_id_list = df_aware_device["device_id"].tolist()
device_id_list_answered_file = "device_id_list_answered.txt"

In [3]:
logging.basicConfig(filename='prepare_info.log',level=logging.INFO)

In [4]:
def find_surveys_in_between(current_time, srvy):
    """
    Find the time slot that current time locates.
    @return: flag: -1: current time not in any of the survey time range;
                    0: current time in several survey time range; need to check time schedule database
                    1: current time in one time range;
    current_time_range: corresponding timerange
    """
    current_time = pd.to_datetime(current_time)
    count = 0
    current_time_range = (-1,{"from":pd.to_datetime("2100-04-08 13:05:04"), "to":pd.to_datetime("1970-04-08 13:05:04")})
    search_flag = -1
    for row in srvy.iterrows():
        if ((current_time >= row[1]["from"]) & (current_time <= row[1]["to"])):
            count += 1
            current_time_range = row
    if (count != 1): # check whether the time falls in multiple 
        if (count == 0):
            search_flag = -1
        else:
            search_flag = 0
    else:
        search_flag = 1
    return search_flag, current_time_range
def extract_response_within_range(current_time, srvy, resps):
    """
    Find the time slot that current time locates. Then extract all resps within this period.
    
    @return: flag: -1: current time not in any of the survey time range;
                    0: current time in several survey time range; need to check time schedule database
                    1: current time in one time range;
    current_time_range: corresponding timerange
    """
    search_flag, current_time_range = find_surveys_in_between(current_time, srvy)
    response_in_range = resps[(resps['startDate'] >= current_time_range[1]['from'])
                              & (resps['endDate'] <= (current_time_range[1]['to'] + pd.Timedelta(minutes = 10)))]
    return search_flag, response_in_range

In [5]:
# set current time
current_time = pd.to_datetime("2018-2-1 21:05:04")
survey_date_file = "./survey_datetimes.csv"

In [22]:
# prepare the time schedule
survey_dates = pd.read_csv(survey_date_file)
survey_dates['from'] = pd.to_datetime(survey_dates['date'] + ' ' + survey_dates['start'],format = "%Y/%m/%d %H:%M")
survey_dates['to'] = pd.to_datetime(survey_dates['date'] + ' ' + survey_dates['expiry'], format = "%Y/%m/%d %H:%M")
survey_dates['date'] = pd.to_datetime(survey_dates['date'], format= "%Y/%m/%d")
survey_dates['start'] = pd.to_datetime(survey_dates['start'],format= '%H:%M').dt.time
survey_dates['expiry'] = pd.to_datetime(survey_dates['expiry'],format= '%H:%M').dt.time

In [30]:
print(survey_dates["date"][0].strftime('%Y/%m/%d'), survey_dates["expiry"][0].strftime('%H:%M'))

2019/01/27 23:59


In [32]:
pd.to_datetime("2018-2-1 21:5:4").strftime('%Y/%m/%d %H:%M')

'2018/02/01 21:05'

In [7]:
# select the time range to determine the survey type, survey link, etc. 
search_flag, current_time_range = find_surveys_in_between(current_time, survey_dates)
if (search_flag == -1):
    logging.info("Current time: {}, not in the schedule list".format(current_time))
    quit()
elif (search_flag == 0):
    logging.error("Current time found multiple entries in the schedule list")
    quit()
else:
    survey_type = current_time_range[1]["type"]
    logging.info("Current time: {0}, found in the schedule. Survey type: {1}".format(current_time, survey_type))

In [10]:
# select the correspnding config file
with open("survey-config-overall.json", "r") as f:
    config_overall = json.load(f)
    
data_file = config_overall[survey_type]["data_file"]
survey_link = config_overall[survey_type]["survey_link"]


In [11]:
responses = pd.read_csv(data_file,skiprows=range(1,2))

In [12]:
def cleaning_up(responses):
    # clean-up - remove invalid responses
    # according to qualtrics (https://goo.gl/p4g16k):
    #    status ==  0 --> normal
    #    status ==  1 --> preview
    #    status ==  2 --> test (NA in our data)
    #    status ==  4 --> imported (NA in our data)
    #    status ==  8 --> spam (e.g. because of duplicate submissions)
    #    status == 16 --> offline (NA in our data)
    valid = responses['status'] == 0
    print('removing {} response(s) with invalid status'.format(responses.shape[0] - sum(valid)))
    responses = responses[valid]

    # clean-up - remove unfinished responses
    # according to qualtrics (https://goo.gl/p4g16k):
    #    finished == 0 --> closed without completion (progress < 100)
    #    finished == 1 --> submitted (progress == 100)
    finished = responses['finished'] == 1
    print('removing {} unfinished response(s)'.format(responses.shape[0] - sum(finished)))
    responses = responses[finished]

    # clean-up - remove near empty responses (threshold as half of the questions)
    empty_count = responses.isnull().sum(axis=1) / float(responses.shape[1])
    empty_count_threshold = 0.5
    print('removing {} near empty response(s)'.format(sum(empty_count >= empty_count_threshold)))
    responses = responses[empty_count < empty_count_threshold]
    
    # conversion - convert the date columns to datetime objects and adjust for time zone different
    columns_date = ["startDate", "endDate", "recordedDate"]
    responses[columns_date] = responses[columns_date].apply(pd.to_datetime)
    # need to check Phase II time zone of the file
    responses[columns_date] = responses[columns_date] + pd.DateOffset(hours=1) # records of phase I are in MT
    # responses.head()
    return responses

In [13]:
responses = cleaning_up(responses)

removing 2 response(s) with invalid status
removing 101 unfinished response(s)
removing 136 near empty response(s)


In [41]:
responses = cleaning_up(responses)

removing 2 response(s) with invalid status
removing 101 unfinished response(s)
removing 127 empty response(s)
remaining empty response(s): 0
removing 7 near empty response(s)
time on survey varies from 55 (sec) to 56262 (sec) (M = 406.819, std = 1763.787, std_err = 26.608)
tagged 48 response(s) longer than two standard deviation of the mean (>= 3934.393 secs)


In [16]:
# get the surveys that are within the current time range
# acceptable submissions are submissions that started on the survey date and after start time and have been recorded
# on the survey date and before the expiry date. Any other submission is late.
search_flag, responses_in_current_time_range = extract_response_within_range(current_time, survey_dates, responses)

In [18]:
current_time

Timestamp('2018-02-01 21:05:04')

In [20]:
response_device_id_list = responses_in_current_time_range["deviceID"].tolist()
not_answered_device_id_list = list(set(device_id_list) - set(response_device_id_list))
with open(device_id_list_file_answered, "w") as f:
    f.write("\n".join(not_answered_device_id_list))
# push_notification(not_answered_device_id_list)

KeyError: 'device_id'